In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/challenges-in-representation-learning-facial-expression-recognition-challenge/icml_face_data.csv
/kaggle/input/challenges-in-representation-learning-facial-expression-recognition-challenge/fer2013.tar.gz
/kaggle/input/challenges-in-representation-learning-facial-expression-recognition-challenge/example_submission.csv
/kaggle/input/challenges-in-representation-learning-facial-expression-recognition-challenge/train.csv
/kaggle/input/challenges-in-representation-learning-facial-expression-recognition-challenge/test.csv


In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
df = pd.read_csv('../input/challenges-in-representation-learning-facial-expression-recognition-challenge/train.csv')
t = []

In [4]:
for i in range(len(df)):
    t.append(np.reshape(np.array(df['pixels'][i].split(' ')).astype(float),(48,48,1)))
df

,emotion,pixels
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...
1,0,151 150 147 155 148 133 111 140 170 174 182 15...
2,2,231 212 156 164 174 138 161 173 182 200 106 38...
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...
...,...,...
28704,2,84 85 85 85 85 85 85 85 86 86 86 87 86 86 91 9...
28705,0,114 112 113 113 111 111 112 113 115 113 114 11...
28706,4,74 81 87 89 95 100 98 93 105 120 127 133 146 1...
28707,0,222 227 203 90 86 90 84 77 94 87 99 119 134 14...


In [5]:
t = np.array(t)
emotion_one_hot = []
for i in df['emotion']:
    emotion_one_hot.append(np.zeros((7)))
    emotion_one_hot[-1][i] = 1.0
emotion_one_hot = np.array(emotion_one_hot)
emotion_one_hot.shape

(28709, 7)

In [6]:
t /= 255.0
t.shape

(28709, 48, 48, 1)

In [7]:
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [8]:
with tpu_strategy.scope():
    model = Sequential()
    model.add(layers.Conv2D(filters=32, kernel_size=(3,3), input_shape=(48,48,1), activation='relu'))
    model.add(layers.ZeroPadding2D((1,1)))
    model.add(layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu'))
    model.add(layers.ZeroPadding2D((1,1)))
    model.add(layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu'))
    model.add(layers.ZeroPadding2D((1,1)))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(pool_size=(2,2)))

    model.add(layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu'))
    model.add(layers.ZeroPadding2D((1,1)))
    model.add(layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu'))
    model.add(layers.ZeroPadding2D((1,1)))
    model.add(layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu'))
    model.add(layers.ZeroPadding2D((1,1)))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(pool_size=(2,2)))

    model.add(layers.Conv2D(filters=128, kernel_size=(3,3), activation='relu'))
    model.add(layers.ZeroPadding2D((1,1)))
    model.add(layers.Conv2D(filters=128, kernel_size=(3,3), activation='relu'))
    model.add(layers.ZeroPadding2D((1,1)))
    model.add(layers.Conv2D(filters=128, kernel_size=(3,3), activation='relu'))
    model.add(layers.ZeroPadding2D((1,1)))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(pool_size=(2,2)))

    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(7, activation='softmax'))
    model.compile(metrics=['accuracy'],optimizer='adam',loss='categorical_crossentropy')

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 46, 46, 32)        320       
_________________________________________________________________
zero_padding2d (ZeroPadding2 (None, 48, 48, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 46, 46, 32)        9248      
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 48, 48, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 46, 46, 32)        9248      
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 48, 48, 32)        1

In [10]:
model.fit(t,emotion_one_hot,batch_size = 16 * tpu_strategy.num_replicas_in_sync, epochs=20)

Epoch 1/20
225/225 [==============================] - 7s 30ms/step - loss: 1.8160 - accuracy: 0.2978
Epoch 2/20
225/225 [==============================] - 4s 17ms/step - loss: 1.4689 - accuracy: 0.4285
Epoch 3/20
225/225 [==============================] - 4s 16ms/step - loss: 1.3053 - accuracy: 0.5002
Epoch 4/20
225/225 [==============================] - 4s 18ms/step - loss: 1.1911 - accuracy: 0.5472
Epoch 5/20
225/225 [==============================] - 4s 16ms/step - loss: 1.1023 - accuracy: 0.5846
Epoch 6/20
225/225 [==============================] - 4s 17ms/step - loss: 1.0254 - accuracy: 0.6133
Epoch 7/20
225/225 [==============================] - 4s 17ms/step - loss: 0.9504 - accuracy: 0.6410
Epoch 8/20
225/225 [==============================] - 4s 16ms/step - loss: 0.8757 - accuracy: 0.6728
Epoch 9/20
225/225 [==============================] - 4s 16ms/step - loss: 0.7911 - accuracy: 0.7066
Epoch 10/20
225/225 [==============================] - 4s 16ms/step - loss: 0.6834 - accura

In [11]:
model.save('model.h5')

In [12]:
os.chdir(r'/kaggle/working')

In [13]:
!zip -r file.zip './'

  adding: __notebook__.ipynb (deflated 96%)
  adding: model.h5 (deflated 7%)


In [14]:
from IPython.display import FileLink
FileLink('./file.zip')

/kaggle/working/file.zip